In [1]:
%pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from process_data import  get_df
import numpy as np
import seaborn as sns
pd.set_option('display.float_format', lambda x: '%.0f' % x)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



In [2]:
df = get_df()
df

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,13114483460844900352,4/4/2019 12:01,770,4/25/2019 14:35,662,1823652,208201448079117,33664962239,35521209507511,9.16456699548519E+015,...,15854611,2501332,8198936,9656251,278082303,14344150,171744450,8814393,36749741,308879636
1,13114483482878900224,4/9/2019 13:04,235,4/25/2019 8:15,606,1365104,208201909211140,33681854413,35794009006359,L77566A,...,20247395,19111729,18338413,17227132,608750074,1170709,526904238,15055145,53800391,653384965
2,13114483484080500736,4/9/2019 17:42,1,4/25/2019 11:58,652,1361762,208200314458056,33760627129,35281510359387,D42335A,...,19725661,14699576,17587794,6163408,229584621,395630,410692588,4215763,27883638,279807335
3,13114483485442799616,4/10/2019 0:31,486,4/25/2019 7:36,171,1321509,208201402342131,33750343200,35356610164913,T21824A,...,21388122,15146643,13994646,1097942,799538153,10849722,749039933,12797283,43324218,846028530
4,13114483499480700928,4/12/2019 20:10,565,4/25/2019 10:40,954,1089009,208201401415120,33699795932,35407009745539,D88865A,...,15259380,18962873,17124581,415218,527707248,3529801,550709500,13910322,38542814,569138589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149996,7277825670196679680,4/29/2019 7:28,451,4/30/2019 6:02,214,81230,208202201200072,33650688697,35483109451938,D20434A,...,16191667,11763428,17883703,19678161,526609673,9197207,3264510,13487416,57628851,574175259
149997,7349883264234609664,4/29/2019 7:28,483,4/30/2019 10:41,187,97970,208201908153249,33663449963,35660508296467,D10223C,...,13877234,8288284,19350146,21293148,626893062,4735033,712180387,2457758,39135081,666648844
149998,13114483573367300096,4/29/2019 7:28,283,4/30/2019 10:46,810,98249,208201711161187,33621890103,35721209870907,T51102A,...,22660510,1855903,9963942,5065760,553539484,13394316,121100856,11314729,34912224,592786405
149999,13114483573367300096,4/29/2019 7:28,696,4/30/2019 10:40,327,97910,208202101098075,33619622058,86186204011457,L88342B,...,8817106,8305402,3322253,13172589,352536971,2529475,814713113,1406930,29626096,371895920


In [ ]:
# Start by identifying the top 10 handsets used by the customers
Handset_counts = df['Handset Type'].value_counts()
Handset_counts = Handset_counts .reset_index()
Handset_counts.columns = ['Handset Type', 'Count']
Handset_counts.head(10)

In [ ]:
# identify the top 3 handset manufacturers
manufacturers_counts = df['Handset Manufacturer'].value_counts()
manufacturers_counts = manufacturers_counts .reset_index()
manufacturers_counts.columns = ['top 3 handset manufacturers', 'Count']
manufacturers_counts.head(3)

In [ ]:
df.columns

In [ ]:
# identify the top 5 handsets per top 3 handset manufacturer
# first filter the data 
filtered_df = df[df['Handset Manufacturer'].isin(['Apple', 'Samsung', 'Huawei'])]
filtered_df.iloc[0]

In [ ]:
# identify the top 5 handsets per top 3 handset manufacturer
Handset_counts = filtered_df['Handset Type'].value_counts()
Handset_counts = Handset_counts .reset_index()
Handset_counts.columns = ['top 5 handsets', 'Count']
Handset_counts.head(5)

In [ ]:
# Aggregate per user the following information in the column  
# number of xDR sessions
# Session duration
# the total download (DL) and upload (UL) data
# the total data volume (in Bytes) during this session for each application

# Convert 'Start' and 'End' columns to datetime format for better handling
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])

# Calculate Session Duration in seconds
df['Session Duration (s)'] = (df['End'] - df['Start']).dt.total_seconds()

# List of applications for which you want to calculate total data volume
applications = ['Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other']

# Aggregate per user
user_aggregated = df.groupby('MSISDN/Number').agg({
    'Bearer Id': 'count',  # Number of xDR sessions
    'Session Duration (s)': 'sum',  # Total session duration
    'Total DL (Bytes)': 'sum',  # Total download data
    'Total UL (Bytes)': 'sum',  # Total upload data
    **{f'{app} DL (Bytes)': 'sum' for app in applications},  # Total download data for each application
    **{f'{app} UL (Bytes)': 'sum' for app in applications}  # Total upload data for each application
})

# Concatenate download and upload columns for each application
for app in applications:
    user_aggregated[f'{app} (Total Bytes)'] = user_aggregated[f'{app} DL (Bytes)'] + user_aggregated[f'{app} UL (Bytes)']
    user_aggregated.drop([f'{app} DL (Bytes)', f'{app} UL (Bytes)'], axis=1, inplace=True)

# Rename columns for clarity
user_aggregated.rename(columns={'Bearer Id': 'Number of xDR sessions'}, inplace=True)

# Display the aggregated information
user_aggregated.head(10)

In [ ]:
missing_values = df.isnull()
missing_values


In [ ]:
missing_values_count = df.isnull().sum()
missing_values_count

In [ ]:
#  Conduct an exploratory data analysis on those data & communicate useful insights. Ensure that you 
# identify and treat all missing values and outliers in the dataset by replacing by the mean of the corresponding column.

non_numeric_columns = df.select_dtypes(exclude=['number']).columns

df_numeric = df.drop(non_numeric_columns, axis=1)

cleaned_data = df_numeric.fillna(df_numeric.mean())


In [ ]:
# Analyze the basic metrics (mean, median, etc) in the Dataset (explain) & their importance for the global objective.
selected_columns = [
    "Dur. (ms)",
    "Avg Bearer TP DL (kbps)",
    "Avg Bearer TP UL (kbps)",
    "Social Media DL (Bytes)",
    "Social Media UL (Bytes)",
    "Google DL (Bytes)",
    "Google UL (Bytes)",
    "Email DL (Bytes)",
    "Email UL (Bytes)",
    "Youtube DL (Bytes)",
    "Youtube UL (Bytes)",
    "Netflix DL (Bytes)",
    "Netflix UL (Bytes)",
    "Gaming DL (Bytes)",
    "Gaming UL (Bytes)",
    "Total UL (Bytes)",
    "Total DL (Bytes)",
    "Other DL (Bytes)",
    "Other UL (Bytes)"
]

# Create a new DataFrame with selected columns
displayed_data = cleaned_data[selected_columns]

# Display the new DataFrame
displayed_data.describe()


In [ ]:

# Conduct a Non-Graphical Univariate Analysis by computing dispersion parameters for each quantitative 
# variable and provide useful interpretation. 

quantitative_columns = displayed_data.select_dtypes(include=['number']).columns

dispersion_data = pd.DataFrame(index=quantitative_columns, columns=['Range', 'Variance', 'Standard Deviation', 'IQR'])

for column in quantitative_columns:
    data = displayed_data[column].dropna() 
   
    data_range = data.max() - data.min()
    
    data_variance = data.var()
    
    data_std_dev = data.std()
    
    # Interquartile Range (IQR)
    data_iqr = data.quantile(0.75) - data.quantile(0.25)
    
    # Store values in the DataFrame
    dispersion_data.loc[column] = [data_range, data_variance, data_std_dev, data_iqr]

dispersion_data

In [ ]:
import matplotlib.pyplot as plt

# Assuming df is your DataFrame
numeric_columns = displayed_data.select_dtypes(include='number').columns

for column in numeric_columns:
    plt.figure(figsize=(8, 5))
    plt.hist(df[column].dropna(), bins=30, color='skyblue', edgecolor='black')
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

# List of applications
applications = [
    'Social Media',
    'Google',
    'Email',
    'Youtube',
    'Netflix',
    'Gaming',
    'Other',
]

# Calculate total DL+UL data
cleaned_data['Total Data (DL+UL)'] = cleaned_data['Total UL (Bytes)'] + cleaned_data['Total DL (Bytes)']

# Create scatter plots for each application vs. Total Data (DL+UL)
plt.figure(figsize=(14, 10))
for app in applications:
    sns.scatterplot(x=cleaned_data[app + ' DL (Bytes)'], y=cleaned_data[app + ' UL (Bytes)'], label=app)

plt.xlabel('DL Data (Bytes)')
plt.ylabel('UL Data (Bytes)')
plt.title('Scatter Plot of DL vs. UL Data for Each Application')
plt.legend()
plt.show()

In [ ]:
# Convert 'Start' and 'End' columns to datetime format for better handling

df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])

# Calculate Session Duration in seconds
df['Session Duration (s)'] = (df['End'] - df['Start']).dt.total_seconds()

# Calculate total DL+UL data
df['Total Data (DL+UL)'] = df['Total UL (Bytes)'] + df['Total DL (Bytes)']

# Group users into deciles based on total session duration
df['Duration Decile'] = pd.qcut(df.groupby('MSISDN/Number')['Session Duration (s)'].transform('sum'), q=10, labels=False, duplicates='drop')

# Group by duration decile and compute total data per decile
decile_data = df.groupby('Duration Decile')['Total Data (DL+UL)'].sum().reset_index()

# Sort deciles by total data in descending order
decile_data = decile_data.sort_values(by='Total Data (DL+UL)', ascending=False)

# Display the total data per decile
print(decile_data)

In [ ]:

# List of columns for correlation analysis
columns_of_interest = [
    'Social Media DL (Bytes)',
    'Google DL (Bytes)',
    'Email DL (Bytes)',
    'Youtube DL (Bytes)',
    'Netflix DL (Bytes)',
    'Gaming DL (Bytes)',
    'Other DL (Bytes)',
    'Social Media UL (Bytes)',
    'Google UL (Bytes)',
    'Email UL (Bytes)',
    'Youtube UL (Bytes)',
    'Netflix UL (Bytes)',
    'Gaming UL (Bytes)',
    'Other UL (Bytes)',
]

# Subset the DataFrame with the specified columns
correlation_data = cleaned_data[columns_of_interest]

# Calculate correlation matrix
correlation_matrix = correlation_data.corr()

# Display the correlation matrix

correlation_matrix

In [ ]:
# Select relevant columns
app_columns = ['Social Media DL (Bytes)', 'Social Media UL (Bytes)',
               'Google DL (Bytes)', 'Google UL (Bytes)',
               'Email DL (Bytes)', 'Email UL (Bytes)',
               'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
               'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
               'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
               'Other DL (Bytes)', 'Other UL (Bytes)']

# Create new columns for total bytes for each application
for app in ['Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other', 'Total']:
    cleaned_data[f'{app} Total Bytes'] = cleaned_data[f'{app} DL (Bytes)'] + cleaned_data[f'{app} UL (Bytes)']

# Select only the new total bytes columns
total_bytes_columns = [f'{app} Total Bytes' for app in ['Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other']]
total_bytes_data = cleaned_data[total_bytes_columns]

# Calculate correlation matrix for total bytes
corr_matrix_total_bytes = total_bytes_data.corr()

# Plot heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix_total_bytes, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix for Total Bytes')
plt.show()


In [ ]:


# Select the relevant columns for PCA
all_columns_for_pca = ['Social Media DL (Bytes)', 'Social Media UL (Bytes)',
                       'Google DL (Bytes)', 'Google UL (Bytes)',
                       'Email DL (Bytes)', 'Email UL (Bytes)',
                       'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
                       'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
                       'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
                       'Other DL (Bytes)', 'Other UL (Bytes)',
                       'Total DL (Bytes)', 'Total UL (Bytes)',
                       'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
                       'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
                       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
                       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
                       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
                       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
                       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
                       'HTTP DL (Bytes)', 'HTTP UL (Bytes)',
                       'Activity Duration DL (ms)', 'Activity Duration UL (ms)',
                       'Nb of sec with 125000B < Vol DL', 'Nb of sec with 1250B < Vol UL < 6250B',
                       'Nb of sec with 31250B < Vol DL < 125000B', 'Nb of sec with 37500B < Vol UL',
                       'Nb of sec with 6250B < Vol DL < 31250B', 'Nb of sec with 6250B < Vol UL < 37500B',
                       'Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1250B']

# Create a subset dataframe with relevant columns
data_for_pca = cleaned_data[all_columns_for_pca]

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_pca)

# Apply PCA
pca = PCA(10)
principal_components = pca.fit_transform(scaled_data)

# Get explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Plot explained variance
plt.figure(figsize=(12, 6))
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.8, align='center')
plt.step(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio.cumsum(), where='mid')
plt.title('Explained Variance vs. Number of Principal Components')
plt.xlabel('Number of Principal Components')
plt.ylabel('Explained Variance Ratio')
plt.show()

principal_components  = pd.DataFrame(principal_components)
principal_components

In [ ]:
! pip install scikit-learn
